In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep
import random
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from tqdm import tqdm

In [2]:
main_link = 'https://www.lazada.vn/catalog/?q=Trang%20s%E1%BB%A9c'
path = 'chromedriver.exe'

# # Customize chrome displayLa
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get(main_link)


In [3]:
#Crawl the information displayed on the outside of the page
def get_Data_On_Page():
    #The information is definitely there
    Type = [elem.get_attribute("value") for elem in driver.find_elements(By.CSS_SELECTOR, ".search-box__input--O34g")]
    Title = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")]
    Link = [elem.get_attribute('href') for elem in driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")]
    Price_sale = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".ooOxS")]
    
    #The information may or may not be available
    #If not, assign None
    Sale_off, Total_sold, Preview = [], [], []
    for i in range(1, len(Title)+1): #/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[1]/div/div/div[2]/div[4]/span
        try:
            one_sale_off = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[4]/span".format(i))
            Sale_off.append(one_sale_off.text)
        except NoSuchElementException:
            Sale_off.append(None)

        try:
            one_total_sold = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/span[1]/span[1]".format(i))
            Total_sold.append(one_total_sold.text)
        except NoSuchElementException:
            Total_sold.append(None)

        try:
            one_preview = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/div/span".format(i))
            Preview.append(one_preview.text)
        except NoSuchElementException:
            Preview.append(None)
    
    Location = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".oa6ri")]
    if len(Type) == 1:
        Type = Type * len(Title)

    #Create a dictionary with the lists
    data = {
        "Type": Type,
        "Title": Title,
        "Link": Link,
        "Price_sale": Price_sale,
        "Sale_off": Sale_off,
        "Total_sold": Total_sold,
        "Preview": Preview,
        "Location": Location
    }

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data)
    return df

In [4]:
#Call the function and get the DF
df_Jewelry = get_Data_On_Page()
df_Jewelry.index = np.arange(1, len(df_Jewelry)+1)
print(df_Jewelry)

         Type                                              Title  \
1   Trang sức  17km pha lê May Mắn Vòng đeo tay đính hạt cho ...   
2   Trang sức  🔥 Vòng kim cương mạ bạc đơn giản S925 dành cho...   
3   Trang sức  【YIDEA HONGKONG】1pc nhẫn Cặp Đôi Đồng Tâm Thắt...   
4   Trang sức  Vòng Cổ Dây Chuyền Chạm Xương Quai Xanh Hình T...   
5   Trang sức  JWER Quà Tặng Trang Sức Cặp Đôi Hình Trái Tim ...   
6   Trang sức  Nhẫn Chống Gỉ Cho Nữ Nhẫn Trang Sức Kim Cương ...   
7   Trang sức  Nước rửa bạc, nước rửa trang sức vàng bạc 50ml...   
8   Trang sức  BESFOME Bộ Ba Phần Của Các Vụ Nổ Phát Sóng Trự...   
9   Trang sức  18K vàng sang trọng Rhinestone Nhẫn Zircon cho...   
10  Trang sức  Lấp lánh hơn cả ánh đèn đêm: Bộ Trang Sức 5 mó...   
11  Trang sức  【COMBO 5 HỘP】Nước rửa trang sức vàng bạc, nước...   
12  Trang sức   Nhẫn trơn khắc nổi mạ vàng 24k trang sức sỉ Kami   
13  Trang sức                        bộ trang sức nữ hình cầu bi   
14  Trang sức  [HCM]Nước rửa bạc, nước rửa trang

In [5]:
#Get the link and access each product to get more features
def get_Data_OneProduct_Link(link):
    driver.get(link)                            #Get link
    sleep(random.uniform(3, 5))                 #Random sleep to mimic human behavior and avoid getting blocked
    
    Price_original = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".notranslate.pdp-price.pdp-price_type_deleted.pdp-price_color_lightgray.pdp-price_size_xs")]
    Ship_price = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".delivery-option-item.delivery-option-item_type_standard .delivery-option-item__body .delivery-option-item__shipping-fee.no-subtitle")]
    Return = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".delivery-option-item.delivery-option-item_type_returnPolicy30 .delivery-option-item__body .delivery-option-item__info .delivery-option-item__title")]

    # Sale_rating & Ship_on_time & Chat_respone
    percentage = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".seller-info-value ")]
    Sale_rating = percentage[0] if len(percentage) > 0 else None
    Ship_on_time = percentage[1] if len(percentage) > 1 else None
    Chat_response = percentage[2] if len(percentage) > 2 else None


    #Cuộn trang từ từ để tải phần đánh giá sao
    while True:
        driver.execute_script("window.scrollBy(0, 400);")
        sleep(1)  #Đợi một chút để nội dung tải
        try:
            stars = driver.find_elements(By.CSS_SELECTOR, ".detail .percent")
            if len(stars) >= 5:
                break
        except NoSuchElementException:
            continue
    
    One_star, Two_star, Three_star, Four_star, Five_star = [], [], [], [], []
    stars = [elem.text for elem in stars]

    One_star.append(stars[4] if len(stars) > 4 else 'N/A')
    Two_star.append(stars[3] if len(stars) > 3 else 'N/A')
    Three_star.append(stars[2] if len(stars) > 2 else 'N/A')
    Four_star.append(stars[1] if len(stars) > 1 else 'N/A')
    Five_star.append(stars[0] if len(stars) > 0 else 'N/A')


    #Create a dictionary with the lists
    One_Product = {
        "Price_original": Price_original[0] if Price_original else None,
        "Ship_price": Ship_price[0] if Ship_price else None,
        "Return": Return[0] if Return else None,
        "Sale_rating": Sale_rating,
        "Ship_on_time": Ship_on_time,
        "Chat_response": Chat_response,
        "One_star": One_star,
        "Two_star": Two_star,
        "Three_star": Three_star,
        "Four_star": Four_star,
        "Five_star": Five_star
    }
    
    return One_Product 

In [6]:
#Initialize dictionary to store details for all products
Products = {
    "Price_original": [],
    "Ship_price": [],
    "Return": [],
    "Sale_rating": [],
    "Ship_on_time": [],
    "Chat_response": [],
    "One_star": [],
    "Two_star": [],
    "Three_star": [],
    "Four_star": [],
    "Five_star": []
}

#Create a progress bar to track the crawl process
for link in tqdm(df_Jewelry.Link, desc="Crawling link product"):
    LinkProduct = get_Data_OneProduct_Link(link)
    for key in Products.keys():
        Products[key].append(LinkProduct[key])

Crawling link product: 100%|██████████| 40/40 [20:16<00:00, 30.42s/it]


In [7]:
#Add columns to dienthoai
for key, value in Products.items():
    df_Jewelry[key] = value

In [8]:
print(df_Jewelry)

         Type                                              Title  \
1   Trang sức  17km pha lê May Mắn Vòng đeo tay đính hạt cho ...   
2   Trang sức  🔥 Vòng kim cương mạ bạc đơn giản S925 dành cho...   
3   Trang sức  【YIDEA HONGKONG】1pc nhẫn Cặp Đôi Đồng Tâm Thắt...   
4   Trang sức  Vòng Cổ Dây Chuyền Chạm Xương Quai Xanh Hình T...   
5   Trang sức  JWER Quà Tặng Trang Sức Cặp Đôi Hình Trái Tim ...   
6   Trang sức  Nhẫn Chống Gỉ Cho Nữ Nhẫn Trang Sức Kim Cương ...   
7   Trang sức  Nước rửa bạc, nước rửa trang sức vàng bạc 50ml...   
8   Trang sức  BESFOME Bộ Ba Phần Của Các Vụ Nổ Phát Sóng Trự...   
9   Trang sức  18K vàng sang trọng Rhinestone Nhẫn Zircon cho...   
10  Trang sức  Lấp lánh hơn cả ánh đèn đêm: Bộ Trang Sức 5 mó...   
11  Trang sức  【COMBO 5 HỘP】Nước rửa trang sức vàng bạc, nước...   
12  Trang sức   Nhẫn trơn khắc nổi mạ vàng 24k trang sức sỉ Kami   
13  Trang sức                        bộ trang sức nữ hình cầu bi   
14  Trang sức  [HCM]Nước rửa bạc, nước rửa trang

In [9]:
#Convert to csv
Data_Jewelry_Page01 = df_Jewelry.to_csv("Data_Jewelry_Page01.csv", index=False)